### Lendo todos os arquivos csv do diretório e guardar em um objeto

```python

    Primeiramente vamos ler todos arquivos xlsx que estão presentes em um diretório cujo nome. 
    Então, criamos uma função anônima (lambda) para varrer todos os arquivos xlsx e restringimos a nossa condição de busca ao 
```

In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import os
diretorio = r'./ESTABELECIMENTOSCSV/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

### Realizando a leitura para cada arquivo
```python
        Podemos fazer a leitura de todos os arquivos presentes no diretório criando uma lista vazia (full_dataset) e utilizamos um loop para fazer a leitura de cada arquivo e, assim preenchermos a lista utilizando o método append.
```

In [2]:

CNPJ = {"EMPRE" : ['CNPJ_BASE', 'RAZAO_SOCIAL', 'NATUREZA_JURIDICA' , 'QUALIFICACAO' ,
          'VALOR_CAPITAL_SOCIAL' , 'PORTE_EMPRESA'  , 'ENTE_FEDERATIVO'],

"ESTABELE" : ['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV' , 'MATRIZ_FILIAL'  , 'NOME_FANTASIA' , 'SITUACAO_CADASTRAL' ,
              'DATA_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  , 'CIDADE_EXTERIOR'  , 'PAIS'  , 'DATA_INICIO_ATIVIDADE'  ,
              'CNAE_PRINCIPAL'  , 'CNAE_SECUNDARIO' , 'TIPO_LOGRADOURO'  , 'LOGRADOURO'  , 'NUMERO'  , 'COMPLEMENTO' ,
              'BAIRRO'  , 'CEP'  , 'UF'  , 'MUNICIPIO'  , 'DDD1'  , 'TELEFONE1'  , 'DDD2'  , 'TELEFONE2'  ,
              'DDD_FAX'  , 'FAX'  , 'EMAIL'  , 'SITUACAO_ESPECIAL'  , 'DATA_SITUACAO_ESPECIAL'],  

"SIMPLES" : ['CNPJ_BASE'  , 'OPCAO_SIMPLES'  , 'DATA_OPCAO_SIMPLES'  , 'DATA_EXCLUSAO_SIMPLES'  ,
             'OPCAO_MEI'  , 'DATA_OPCAO_MEI'  , 'DATA_EXCLUSAO_MEI'],

"SOCIO" : ['CNPJ_BASE'  , 'TIPO'  , 'NOME'  , 'CPF_CNPJ'  , 'QUALIFICACAO'  , 'DATA_ENTRADA' ,
            'PAIS' , 'REPRESENTANTE'  , 'NOME_REPRESENTANTE'  , 'QUALIFICACAO_REPRESENTANTE'  , 'FAIXA_ETARIA'  ],

"PAIS" : ['CODIGO_PAIS' , 'PAIS'],

"MUNIC" : ['MUNICIPIO'  , 'MUNICIPIO'  ],

"QUALS" : ['QUALIFICACAO', 'QUALIFICACAO' ],

"NATJU" : ['NATUREZA_JURIDICA'  , 'NATUREZA_JURIDICA'],

"MOTI" : ['MOTIVO_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  ],

"CNAE" : ['CODIGO_CNAE'  , 'CNAE' ]}


In [3]:
from descompactador import descompactador
from convertearquivo import convertearquivo

descompactador(r'./ESTABELECIMENTOSZIP/', r'./ESTABELECIMENTOSCSV')
convertearquivo(r'./ESTABELECIMENTOSCSV/', 'ESTABELE','.csv')

PermissionError: [Errno 13] Permission denied: './ESTABELECIMENTOSZIP//'

In [4]:
# realizar a leitura para cada arquivo
import pandas as pd

full_dataset = []
for arq in all_files:
    frags = arq.split('.')
    frame = frags[1]
    globals()[f"df_{frame}"] = pd.read_csv(f'{diretorio}{arq}',sep=';' ,encoding="ISO-8859-1", nrows=50, names=CNPJ['ESTABELE'])
    #data = pd.read_csv(f'{diretorio}{arq}',sep=';' ,encoding="ISO-8859-1", nrows=50, names=CNPJ['ESTABELE'])
    full_dataset.append(globals()[f"df_{frame}"])
    #full_dataset.append(data)    
    globals()[f"df_{frame}"].head(1)

In [5]:
print(full_dataset)

[    cnpj_base  cnpj_ordem  cnpj_dv  matriz_filial  \
0    28678846           1        2              1   
1    26754771           1       68              1   
2    37962505           1       47              1   
3    26755226           1       96              1   
4    26755456           1       55              1   
5    26755739           1        5              1   
6    39533174           1       55              1   
7    28679240           1       83              1   
8    26530148           1       21              1   
9    37963714           1        5              1   
10   26756614           1       91              1   
11    9630941           2       64              2   
12   28679430           1        9              1   
13   28679672           1       94              1   
14   32951535          80       38              2   
15   39534173           1       25              1   
16   29773932           2       30              2   
17   37895304           1       74           

### Estabelecendo a conexão e criando o banco

```python
    Vamos criar uma conexão com o SQLite. 
    O único parâmetro que vamos passar é o nome do banco que será criado.

```

In [9]:
import sqlite3
# estabelecendo a conexão e criando o banco
con = sqlite3.connect('database_estabelecimento_sodexo.db')

### Obtendo os nomes das tabelas
        
```python
        Para obter o nome das nossas tabelas utilizamos como referência o nome dos arquivos xlsx. 
        O método splitext é o responsável por separar entre o nome do arquivo e a extensão. 
        O valor cujo índice é [0] garante que pegaremos apenas a primeira coluna com os nomes dos arquivos.
    
```

In [10]:
# obtendo os nomes das tabelas
table_names = [os.path.splitext(arq)[0] for arq in all_files]

### Obtendo os campos de todas as tabelas
    
```python
        Em seguida, obteremos os campos de todas as tabelas. Dentro do laço, coletamos a primeira linha (cabeçalho) e armazenamos em columnNames finalizando com um append na lista table_fields.
    
```

In [11]:
# obtendo os campos de todas as tabelas
table_fields = []
for i in range(0,len(table_names)):
    columnNames = list(full_dataset[i].head(0))
    table_fields.append(columnNames)

### Manipulando o BD
```python
    Antes da criação e inserção de dados, é necessário definirmos um cursor para interagir com os registros do BD.
```

In [12]:
import sqlite3
cur = con.cursor()

### Criando as tabelas no SQLite
    
```python
        Nesse momento, criaremos as tabelas dentro do SQlite. 
        Para cada elemento das minhas tabelas do xsxl, executamos uma query para criação das tabelas. 
        Passando para a função execute() o nome da tabela (table_names[item]) bem como os campos que as compõe utilizando o separador vírgula 
    
(‘,’.join(table_fields[item])).
```


In [13]:
# criando as tabelas no SQLite
for item in range(0,len(table_names)):
    cur.execute(f"""CREATE TABLE IF NOT EXISTS {table_names[item] } ( {','.join(table_fields[item])} )""")
    print(f"""CREATE TABLE IF NOT EXISTS {table_names[item] } ( {','.join(table_fields[item])} )""")

CREATE TABLE IF NOT EXISTS November_Estabelecimentos_Sodexo_Atualizado ( Estabelecimento,Endereço,Cidade_UF,Telefone,Latitude,Longitude )


### Varrendo todas as tabelas e para cada uma é realizado a inserção dos dados
```python

        A última etapa é realizar a carga dos registros em todas as tabelas. 
        Isso será feito por um laço de repetição que percorre cada tabela realizando a carga dos registros e em seguida grava a execução utilizando o commit().
```


In [18]:
# Varre todas as tabelas e para cada uma é realizada a inserção dos dados
for i in range(0, len(table_names)):

    query = f"""INSERT INTO  {str(table_names[i])} ( {','.join(table_fields[i])}  ) VALUES ( {','.join(map(str,'?'*len(full_dataset[i].columns)))} )"""


    full_dataset[i] = full_dataset[i].astype(str)

    for j in range(0, len(full_dataset[i])):
        insert_register = tuple(full_dataset[i].iloc[j])
        cur.execute(query, insert_register)
        con.commit()

### Função Select nos dados
    
```python
        Chamando a função com parâmetros CAMPO, TABELA, WHERE, CONDIÇÃO 
    
```

In [32]:
def SELECT_WHERE(campo:str = '*' ,tabela:str = table_names[0], where:str = table_fields[0][2], condicao:str = 'Belo Horizonte, MG'):
    select = f""" SELECT {campo} FROM {tabela}
            WHERE {where} = '{condicao}'
    """

    cur.execute(select)
    rows = cur.fetchall()
    print(len(rows))
    for row in rows:
        print(row)

In [ ]:
SELECT_WHERE()